<a href="https://colab.research.google.com/github/yangou201/Hello-World/blob/master/cifar100_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install efficientnet_pytorch
!pip install torch_optimizer

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp36-none-any.whl size=16031 sha256=5dd45c9eac9f38e0569ad4acdfe19e9c07cd190534c93e5a433f26e1995032b8
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch
     |████████████████████████████████| 51kB 4.0MB/s 


In [3]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import PIL
import numpy as np

from torch import multiprocessing
from torch.utils.data import DataLoader, Dataset
from torch.optim import lr_scheduler

from torchvision import transforms

import efficientnet_pytorch
import torch_optimizer as optim
import time
import tqdm
import os

In [4]:
IMG_SIZE = 299
BATCH_SIZE = 32
NUM_CLASS = 100
NUM_EPOCHS = 5
MAX_STEPS_PER_EPOCH = 2000
LOG_FREQ = 20
NUM_TOP_PREDICTS = 10
IN_KERNEL = os.environ.get('KAGGLE_WORKING_DIR') is not None

### 加载数据集

In [5]:
transforms_list = [
                transforms.Resize((IMG_SIZE,IMG_SIZE)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomChoice([
                    transforms.RandomResizedCrop(IMG_SIZE),
                    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
                    transforms.RandomAffine(degrees=15, translate=(0.2, 0.2),
                                            scale=(0.8, 1.2), shear=15,
                                            resample=Image.BILINEAR)
                ]),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225]),
            ]
transform = transforms.Compose(transforms_list)
train_data = torchvision.datasets.CIFAR100('/drive', train=True, transform= transform, download=True)
val_data = torchvision.datasets.CIFAR100('/drive', train=False, transform= transform)

Extracting /drive/cifar-100-python.tar.gz to /drive


In [6]:
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE,
                              shuffle=True, num_workers=torch.multiprocessing.cpu_count(), drop_last=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE,
                              shuffle=False, num_workers=torch.multiprocessing.cpu_count(), drop_last=True)

In [7]:
for b in train_loader:
    batch = b
    break
batch[0].shape

torch.Size([32, 3, 299, 299])

### 建立评价参数

In [8]:
class AverageMeter:
    ''' Computes and stores the average and current value '''
    def __init__(self) -> None:
        self.reset()

    def reset(self) -> None:
        self.val = 0.0
        self.avg = 0.0
        self.sum = 0.0
        self.count = 0

    def update(self, val: float, n: int = 1) -> None:
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [9]:
def GAP(predicts: torch.Tensor, confs: torch.Tensor, targets: torch.Tensor) -> float:
    ''' Simplified GAP@1 metric: only one prediction per sample is supported '''
    assert len(predicts.shape) == 1
    assert len(confs.shape) == 1
    assert len(targets.shape) == 1
    assert predicts.shape == confs.shape and confs.shape == targets.shape

    _, indices = torch.sort(confs, descending=True)

    confs = confs.cpu().numpy()
    predicts = predicts[indices].cpu().numpy()
    targets = targets[indices].cpu().numpy()

    res, true_pos = 0.0, 0

    for i, (c, p, t) in enumerate(zip(confs, predicts, targets)):
        rel = int(p == t)
        true_pos += rel

        res += true_pos / (i + 1) * rel

    res /= targets.shape[0] # FIXME: incorrect, not all test images depict landmarks
    return res

In [10]:
def radam(parameters, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
    if isinstance(betas, str):
        betas = eval(betas)
    return optim.RAdam(parameters,
                      lr=lr,
                      betas=betas,
                      eps=eps,
                      weight_decay=weight_decay)

### 建立模型

In [11]:
# class MyModel(nn.Module):
#     def __init__(self, depth, num_classes):
#         super(MyModel, self).__init__()
#         self.depth = depth
#         self.base = efficientnet_pytorch.EfficientNet.from_pretrained(f'efficientnet-b{self.depth}')
#         for para in self.base.parameters():
#             para.requires_grad = False
# #         self.avg_pool = nn.AdaptiveAvgPool2d(1)
#         self.output_filter = self.base._fc.in_features
#         self.base._fc = nn.Linear(self.output_filter, num_classes)
#         for para in self.base._fc.parameters():
#             para.requires_grad = True
# #         self.classifier = nn.Linear(2048, num_classes)
#     def forward(self, x):
#         x = self.base(x)
#         return x

In [38]:
from torchvision.models import inception_v3
class MyModel(nn.Module):
    def __init__(self, num_classes, extra):
        super(MyModel, self).__init__()

        base = inception_v3(pretrained=True)
        for para in base.parameters():
            para.requires_grad = False
        in_features = base.fc.in_features
        base.fc = nn.ReLU(nn.Linear(in_features=in_features, out_features= 2048))
        self.base = base
        self.classifier = nn.Linear(in_features=in_features, out_features= num_classes)
        
    def forward(self, x):
        x = self.base(x)
        # x = self.avg_pool(x).squeeze(-1).squeeze(-1)
        x = self.classifier(x.logits)
        return x

In [37]:
model = MyModel(100, 3)
model.cuda()

RuntimeError: ignored

RuntimeError: ignored

In [27]:
device = torch.device('cuda')
model.cuda()

RuntimeError: ignored

In [14]:

def train_step(train_loader, 
                model, 
                criterion, 
                optimizer,
                epoch, 
                lr_scheduler,
                min_loss):
    print(f'epoch {epoch}')
    batch_time = AverageMeter()
    losses = AverageMeter()
    avg_score = AverageMeter()

    model.train()
    num_steps = min(len(train_loader), MAX_STEPS_PER_EPOCH)

    print(f'total batches: {num_steps}')

    end = time.time()
    lr = None

    for i, data in enumerate(train_loader):
        input_, target = data
        batch_size, _, _, _ = input_.shape
        
        output = model(input_.cuda())
        
        loss = criterion(output, target.cuda())
        confs, predicts = torch.max(output.detach(), dim=1)
        avg_score.update(GAP(predicts, confs, target))
        losses.update(loss.data.item(), input_.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        lr = optimizer.param_groups[0]['lr']
        
        batch_time.update(time.time() - end)
        end = time.time()

        if i % LOG_FREQ == 0:
            print(f'{epoch} [{i}/{num_steps}]\t'
                    f'time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                    f'loss {losses.val:.4f} ({losses.avg:.4f})\t'
                    f'GAP {avg_score.val:.4f} ({avg_score.avg:.4f})'
                    + str(lr))
    # save the best model
    val_loss = losses.avg
    if val_loss < min_loss:
        min_loss = val_loss
        print("save model")
        torch.save(model.state_dict(),'model.pth')

    print(f' * average GAP on train {avg_score.avg:.4f}')

In [15]:
def predict(data, model):
    model.eval()

    activation = nn.Softmax(dim=1)
    all_predicts, all_confs, all_targets = [], [], []

    with torch.no_grad():
          input_, target = data, None

          output = model(input_.cuda())
          output = activation(output)

          confs, predicts = torch.topk(output, NUM_TOP_PREDICTS)
          all_confs.append(confs)
          all_predicts.append(predicts)

          if target is not None:
              all_targets.append(target)

    predicts = torch.cat(all_predicts)
    confs = torch.cat(all_confs)
    targets = torch.cat(all_targets) if len(all_targets) else None
    predicts = torch.argmax(predicts, axis=1)
    return predicts, confs, targets

In [16]:
global_start_time = time.time()

model = MyModel(3, 100)
model.cuda()

criterion = nn.CrossEntropyLoss()

optimizer = radam(model.parameters(), lr=1e-3, betas=(0.9,0.999), eps=1e-3, weight_decay=1e-4)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader)*NUM_EPOCHS, eta_min=1e-6)

min_loss = 10000
for epoch in range(1, NUM_EPOCHS + 1):
    print('-' * 50)
    train_step(train_loader, model, criterion, optimizer, epoch, scheduler, min_loss)

# print('inference mode')
# generate_submission(test_loader, model, label_encoder)

--------------------------------------------------
epoch 1
total batches: 1562


RuntimeError: ignored